# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path
from typing import Any, List

import configparser
import numpy as np
import pandas as pd
import subprocess
import sys

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

In [ ]:
#| export
def nb_setup(autoreload:bool = True,   # True to set autoreload in this notebook
             paths:list(Path) = None   # Paths to add to the path environment variable
            ):
    """Use in first cell of notebook to set autoreload, and paths"""
#   Add paths. Default is 'src' if it exists
    if paths is None:
        p = Path('../src').resolve().absolute()
        if p.is_dir():
            paths = [str(p)]
        else:
            paths=[]
    if paths:
        for p in paths:
            sys.path.insert(1, str(p))
        print(f"Added following paths: {','.join(paths)}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          | --- notebooks
          |        | --- current_nb.ipynb
          |        | --- ...
          |
          |--- src
          |     | --- scripts_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`

In [ ]:
#| export
def colab_install_project_code(
    package_name:str # project package name, e.g. git+https://github.com/vtecftwy/metagentools.git@main
):
    """When nb is running on colab, pip install the project code package"""
    try:
        from google.colab import drive
        ON_COLAB = True
        print('The notebook is running on colab')
        print('Installing project code')
        cmd = f"pip install -U {package_name}"
        run(cmd)

    except ModuleNotFoundError:
        ON_COLAB = False
        print('The notebook is running locally, will not automatically install project code')

    return ON_COLAB

When using colab, the project code must be install every time from its github repo or from the Python Package Index (PyPI).

When running locally, the project code should be pre-installed as part of the environment

In [ ]:
colab_install_project_code(package_name='git+https://github.com/vtecftwy/metagentools.git@main')

The notebook is running locally, will not automatically install project code


False

In [ ]:
#| export
def files_in_tree(
    path: str|Path,               # path to the directory to scan  
    pattern: str|None = None      # pattern (glob style) to match in file name to filter the content
):
    """List files in directory and its subdiretories, print tree starting from parent directory"""
    pattern = '*' if pattern is None else f"*{pattern}*"
    parents = [p.name for p in path.parents]
    paths = []
    pad = ' ' * 2
    idx = 0
    print(f"{parents[0]}")
    print(f"{pad}|--{path.name}")
    for f in [p for p in path.glob(pattern) if p.is_file()]:
        paths.append(f)
        print(f"{pad}|{pad*2}|--{f.name} ({idx})")
        idx += 1
    for d in [p for p in path.iterdir() if p.is_dir()]:
        print(f"{pad}|{pad*2}|--{d.name}")
        for f in [p for p in d.glob(pattern) if p.is_file()]:
            paths.append(f)
            print(f"{pad}|{pad*2}|{pad*2}|--{f.name} ({idx})")
            idx += 1
    return paths

In [ ]:
p2dir = Path('').resolve()

files = files_in_tree(p2dir)
print(f"List of {len(files)} files when unfiltered")

ecutilities
  |--nbs-dev
  |    |--0_02_plotting.ipynb (0)
  |    |--2_01_image_utils.ipynb (1)
  |    |--1_01_eda_stats_utils.ipynb (2)
  |    |--0_01_ipython.ipynb (3)
  |    |--.last_checked (4)
  |    |--sidebar.yml (5)
  |    |--1_02_ml.ipynb (6)
  |    |--index.ipynb (7)
  |    |--nbdev.yml (8)
  |    |--9_01_dev_utils.ipynb (9)
  |    |--styles.css (10)
  |    |--_quarto.yml (11)
  |    |--.ipynb_checkpoints
  |    |    |--0_02_plotting-checkpoint.ipynb (12)
  |    |    |--9_01_dev_utils-checkpoint.ipynb (13)
  |    |    |--0_01_ipython-checkpoint.ipynb (14)
  |    |    |--1_01_eda_stats_utils-checkpoint.ipynb (15)
  |    |    |--index-checkpoint.ipynb (16)
  |    |    |--2_01_image_utils-checkpoint.ipynb (17)
  |    |    |--1_02_ml-checkpoint.ipynb (18)
List of 19 files when unfiltered


In [ ]:
files = files_in_tree(p2dir, pattern='ipynb')
print(f"List of {len(files)} files when filtered")

ecutilities
  |--nbs-dev
  |    |--0_02_plotting.ipynb (0)
  |    |--2_01_image_utils.ipynb (1)
  |    |--1_01_eda_stats_utils.ipynb (2)
  |    |--0_01_ipython.ipynb (3)
  |    |--1_02_ml.ipynb (4)
  |    |--index.ipynb (5)
  |    |--9_01_dev_utils.ipynb (6)
  |    |--.ipynb_checkpoints
  |    |    |--0_02_plotting-checkpoint.ipynb (7)
  |    |    |--9_01_dev_utils-checkpoint.ipynb (8)
  |    |    |--0_01_ipython-checkpoint.ipynb (9)
  |    |    |--1_01_eda_stats_utils-checkpoint.ipynb (10)
  |    |    |--index-checkpoint.ipynb (11)
  |    |    |--2_01_image_utils-checkpoint.ipynb (12)
  |    |    |--1_02_ml-checkpoint.ipynb (13)
List of 14 files when filtered


In [ ]:
#| export
def display_mds(
    *strings:str # any number of strings with text in markdown format
):
    """Display one or several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: this is a note'
           )

**bold** and _italic_

- bullet

- bullet

> Note: this is a note

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display one or several DataFrame in a single cell output"""
    for df in dfs:
        display(df)

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,0.606404,1.514441,1.006457,-1.606369,1.656222
1,0.108780,-1.653024,-1.320576,-0.334409,0.398428
2,0.698610,0.630786,-0.094427,-0.850979,0.809060


,0,1,2,3,4,5,6,7,8,9
0,-0.769657,0.699641,0.230334,-1.014137,0.858768,0.096435,-0.479266,-0.504209,0.070906,-0.776876
1,-0.836383,0.830942,-0.953745,-0.669761,-0.126888,-0.176799,-0.418857,0.976723,3.131587,0.810725
2,-2.104103,0.481809,0.556296,1.808602,-0.165974,-0.688305,-0.451795,0.046845,-0.274624,-0.822890


In [ ]:
#| export
def run_cli(cmd:str = 'ls -l'   # command to execute in the cli
           ):
    """Runs a cli command from jupyter notebook and print the shell output message
    
    Uses subprocess.run with passed command to run the cli command"""
    p = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
    print(str(p.stdout, 'utf-8'))

In [ ]:
run_cli('pwd')

/home/vtec/projects/ec-packages/ecutilities/nbs-dev



In [ ]:
#| export
def get_config_value(section:str,                        # section in the configparser cfg file
                     key:str,                            # key in the selected section
                     path_to_config_file:Path|str=None   # path to the cfg file
                    )-> Any :                            # the value corresponding to section>key>value 
    """Returns the value corresponding to the key-value pair in the configuration file (configparser format)"""
    # validate path_to_config_file
    if path_to_config_file is None:
        path_to_config_file = Path('/content/gdrive/MyDrive/private-across-accounts/config-api-keys.cfg')
    if isinstance(path_to_config_file, str): 
        path_to_config_file = Path(path_to_config_file)
    if not path_to_config_file.is_file():
        raise ValueError(f"No file at {path_to_config_file.absolute()}. Check the path")

    configuration = configparser.ConfigParser()
    configuration.read(path_to_config_file)
    return configuration[section][key]

By defaults (`path_to_config_file is None`), it is assumed that the configuration file is located in the `private-accross-accounts directory` on google drive. If not, a path to the file (`Path` or `str`) must be provided.

The configuration file is expected to be in the format used by the standard module `configparser` [documentation](https://docs.python.org/3/library/configparser.html)

```ascii
    [DEFAULT]
    key = value

    [section_name]
    key = value

    [section_name]
    key = value
```

In [ ]:
path2cfg = Path('../config-sample.cfg').resolve()
assert path2cfg.is_file(), f"{path2cfg} is not a file"

with open(path2cfg, 'r') as fp:
    print(fp.read())

[azure]
azure-api-key= dummy_api_key_for_azure

[kaggle]
kaggle_username = not_my_real_kaggle_name
kaggle_key = dummy_api_key_for_kaggle

[wandb]
api_key = dummy_api_key_for_wandb



In [ ]:
value = get_config_value(section='azure', key='azure-api-key', path_to_config_file=path2cfg)
assert value == 'dummy_api_key_for_azure'

In [ ]:
value = get_config_value(section='kaggle', key='kaggle_username', path_to_config_file=path2cfg)
assert value == 'not_my_real_kaggle_name'

In [ ]:
value = get_config_value(section='wandb', key='api_key', path_to_config_file=path2cfg)
assert value == 'dummy_api_key_for_wandb'

In [ ]:
#| hide
nbdev_export()